In [4]:
class Experiment:
    """
    Organise experiments. Store and compute relevant information of performed experiments.

        steps:          Number of time-steps
        resolution:     Spatial resolution [1 / degree]
        params:         Parameters that have default values
    """
    _allExperiments = {}
    def __init__(self, steps, resolution, params = None):
        if params is None:
            params = {'n_proc': 16, 'dt': 2700}
        self.steps      = steps
        self.resolution = resolution
        self.n_proc     = params['n_proc']
        self.dt         = params['dt']
        self.cpu_hours  = None 
        self.get_model_time()
        self.name = 'EXP_%iY_%iM_%iD_%iDEG'%(
            self.run_time['y'],
            self.run_time['mo'] % 12,
            self.run_time['d'] % 30,
            self.resolution
        )

        self._allExperiments[self.name] = self

    def get_model_time(self):
        """ Compute the model time in hours, days, months and years."""
        seconds    = int(  self.steps   * self.dt   )
        minutes    = int(  seconds      / 60.       )
        hours      = int(  minutes      / 60.       )
        days       = int(  hours        / 24.       )
        months     = int(  days         / 30.       )
        years      = int(  months       / 12.       )
        self.run_time   = dict(
            s   = seconds,
            mi  = minutes,
            h   = hours,
            d   = days,
            mo  = months,
            y   = years
        )
    
    def get_cpu_hours(self, other):
        """ Estimate the needed CPU time.

            exp_ref:    Reference experiment for which the CPU-time was observed.

        """
        
        reso_ratio      = self.resolution / other.resolution
        time_ratio      = self.steps / other.steps
        self.cpu_hours  = reso_ratio**3 * other.cpu_hours * time_ratio
    
    def set_cpu_hours(self, cpu_time, unit='mi'):
        """ Setting the CPU-hours.

            cpu_time:   Observed CPU-time
            unit:       Unit in which the CPU-time was passed
        """
        time_dict       = dict(s =3600., mi = 60., h = 1.)
        self.cpu_hours  = cpu_time / time_dict[unit]
    
    @classmethod
    def print_CPU_hours(self):
        from IPython.display import display_markdown
        table       = '\n| Experiment Name | Simulation-time [years] | Resolution [1/degree] | CPU-time [hours]| \n| :- | :- | :- | :- |'
        sum_hours   = 0.0
        for key, instance in self._allExperiments.items():
            if instance.cpu_hours is not None:
                sum_hours = sum_hours + instance.cpu_hours
                vars = (instance.run_time['y'], instance.resolution, instance.cpu_hours)
                table = table + '\n| '+ key + ' |   %i  |   1/%i  |   %.2f  |'%vars
        table = table + '| | | | | \n|   **SUM**    | | | %i |'%(int(sum_hours))
        display_markdown(table, raw=True)
    
    @classmethod
    def clear_allExperiments(self):
        self._allExperiments = {}

In [7]:
Experiment.clear_allExperiments()

ref         = Experiment(11520, 1)
ref.set_cpu_hours(78.)

e_1_400     = Experiment(400 * 11520, 1)
e_1_400.get_cpu_hours(ref) 

e_4_200     = Experiment(200 * 11520, 4)
e_4_200.get_cpu_hours(ref)

e_8_100     = Experiment(100 * 11520, 8)
e_8_100.get_cpu_hours(ref)

e_16_70     = Experiment(70 * 11520, 16)
e_16_70.get_cpu_hours(ref)

In [8]:
Experiment.print_CPU_hours()


| Experiment Name | Simulation-time [years] | Resolution [1/degree] | CPU-time [hours]| 
| :- | :- | :- | :- |
| EXP_1Y_0M_0D_1DEG |   1  |   1/1  |   1.30  |
| EXP_400Y_0M_0D_1DEG |   400  |   1/1  |   520.00  |
| EXP_200Y_0M_0D_4DEG |   200  |   1/4  |   16640.00  |
| EXP_100Y_0M_0D_8DEG |   100  |   1/8  |   66560.00  |
| EXP_70Y_0M_0D_16DEG |   70  |   1/16  |   372736.00  || | | | | 
|   **SUM**    | | | 456457 |